# CosmosDB as a single source

# Prereqs


In [15]:
#!import code/Setup.cs


Installed Packages Azure.AI.OpenAI, 2.0.0-beta.3 Azure.Identity, 1.12.0 Azure.Search.Documents, 11.6.0 CsvHelper, 33.0.1 Microsoft.Azure.Cosmos, 3.44.0-preview.1 Microsoft.Data.Analysis, 0.21.0 System.Linq.Async, 6.0.1

Installed Packages Azure.Monitor.OpenTelemetry.Exporter, 1.3.0 Microsoft.ML.Tokenizers, 0.22.0-preview.24378.1 Microsoft.SemanticKernel, 1.19.0 Microsoft.SemanticKernel.Connectors.AzureCosmosDBNoSQL, 1.19.0-alpha Microsoft.SemanticKernel.Connectors.OpenAI, 1.19.0 Microsoft.SemanticKernel.Planners.OpenAI, 1.19.0-preview

# Vector Store Setup

## Database Configuration: Overview


### **Understand Index Type, Vector Data Type, and Distance Functions**

#### **`Vector Index Type`**

This option determines how vectors are indexed within Cosmos DB to optimize search performance.

- **`flat` Index Type**: Use for low-dimensional, exact searches on smaller datasets.
- **`quantizedFlat` Index Type**: Choose when you need to balance performance and storage with acceptable accuracy loss in high-dimensional data.
- **`diskANN` Index Type**: Opt for large-scale, high-dimensional datasets where approximate searches suffice, and speed is critical.

<details>
<summary>
Options
</summary>

- **`flat`**: Stores vectors alongside other indexed properties without additional indexing structures. Supports up to **505 dimensions**.

  **When to Use:**

  - **Low-dimensional data**: Ideal for applications with vectors up to 505 dimensions.
  - **Exact search requirements**: When you need precise search results.
  - **Small to medium datasets**: Efficient for datasets where the index size won't become a bottleneck.

    **Real-World Scenario:**

    - **Customer Segmentation**: A retail company uses customer feature vectors (age, income, purchase history) with dimensions well below 505 to segment customers. Exact matches are important for targeted marketing campaigns.

- **`quantizedFlat`**: Compresses (quantizes) vectors before indexing, improving performance at the cost of some accuracy. Supports up to **4096 dimensions**.

  **When to Use:**

  - **High-dimensional data with storage constraints**: Suitable for vectors up to 4096 dimensions where storage efficiency is important.
  - **Performance-critical applications**: When reduced latency and higher throughput are needed.
  - **Acceptable accuracy trade-off**: Minor losses in accuracy are acceptable for performance gains.

    **Real-World Scenario:**

    - **Mobile Image Recognition**: An app recognizes objects using high-dimensional image embeddings. Quantization reduces the storage footprint and improves search speed, crucial for mobile devices with limited resources.

- **`diskANN`**: Utilizes the DiskANN algorithm for approximate nearest neighbor searches, optimized for speed and efficiency. Supports up to **4096 dimensions**.

  **When to Use:**

  - **Large-scale, high-dimensional data**: Best for big datasets where quick approximate searches are acceptable.
  - **Real-time applications**: When fast response times are critical.
  - **Scalability needs**: Suitable for applications expected to grow significantly.

  **Real-World Scenario:**

  - **Semantic Search Engines**: A search engine indexes millions of documents using embeddings from language models like BERT (768 dimensions). DiskANN allows users to get fast search results by efficiently handling high-dimensional data.
</details>

---

#### **`Vector Data Type`**

Specifies the data type of the vector components.

- **`float32` Datatype**: Default choice for precision; use when storage is less of a concern.
- **`uint8` and `int8` Datatypes**: Use for storage efficiency, particularly when data can be quantized.

<details>
<summary>Options</summary>

- **`float32`** (default): 32-bit floating-point numbers.

  **When to Use:**

  - **High precision requirements**: Necessary when the application demands precise calculations.
  - **Standard ML embeddings**: Most machine learning models output float32 vectors.

  **Real-World Scenario:**

  - **Scientific Simulations**: In climate modeling, vectors represent complex data where precision is vital for accurate simulations and predictions.

- **`uint8`**: 8-bit unsigned integers.

  **When to Use:**

  - **Memory optimization**: Reduces storage needs when precision can be sacrificed.
  - **Quantized models**: When vectors are output from models that already quantize data.

  **Real-World Scenario:**

  - **Basic Image Features**: Storing color histograms for image retrieval systems, where each bin can be represented with an 8-bit integer.

- **`int8`**: 8-bit integer with potentially specialized encoding (interpretation may vary; assuming it's an 8-bit integer with logarithmic encoding).

  **When to Use:**

  - **Custom quantization schemes**: When using specialized compression techniques that map floating-point values to an 8-bit integer scale.
  - **Edge devices**: Ideal for applications on devices with extreme memory limitations.

  **Real-World Scenario:**

  - **Audio Fingerprinting**: Compressing audio feature vectors for song recognition apps where storage and quick retrieval are essential.
</details>

---
#### **`Dimension Size`**

The length of the vectors being indexed. Ranges from 0-4096, default is **1536**.
<details>
<summary>Options</summary>


**When to Consider Lower Dimensions (≤ 505):**

  - **Simpler models**: Applications using basic embeddings or feature vectors.
  - **Flat index type**: Required when using the `flat` index type due to its dimension limit.

  *Real-World Scenario:*

  - **Keyword Matching**: Using low-dimensional TF-IDF vectors for document similarity in a content management system.

  **When to Consider Higher Dimensions (506 - 4096):**

  - **Complex models**: Deep learning applications with high-dimensional embeddings.
  - **Advanced search features**: When richer representations of data are necessary for accuracy.

  *Real-World Scenario:*

  - **Face Recognition**: Using high-dimensional embeddings (e.g., 2048 dimensions) to represent facial features for security systems.
</details>

---

#### **`Distance Function`**

Determines how similarity between vectors is calculated. Select based on the nature of similarity in your application—`cosine` for orientation, `dot product` when magnitude matters, and `euclidean` for spatial relevance.

<details>
<summary>Options</summary>

- **`cosine`**: Measures the cosine of the angle between vectors.

  **When to Use:**

  - **Orientation-focused similarity**: When the magnitude is less important than the direction.
  - **Normalized data**: Ideal when vectors are normalized to unit length.

  **Real-World Scenario:**

  - **Document Similarity**: In text analytics, comparing documents based on topic similarity where word counts are normalized.

- **`dot product`**: Computes the scalar product of two vectors.

  **When to Use:**

  - **Magnitude matters**: When both direction and magnitude are significant.
  - **Machine learning models**: Often used in recommendation systems where strength of preferences is important.

  **Real-World Scenario:**

  - **Personalized Recommendations**: Matching users to products by calculating the dot product of user and item embeddings in a collaborative filtering system.

- **`euclidean`**: Calculates the straight-line distance between vectors.

  **When to Use:**

  - **Spatial distance relevance**: When physical distance correlates with similarity.
  - **High-dimensional data**: Suitable for embeddings where both magnitude and direction impact similarity.

  **Real-World Scenario:**

  - **Anomaly Detection**: Identifying outliers in network traffic patterns by measuring Euclidean distances in feature space.

---

### **Option Combinations and Preferred Use-Cases**



#### **Combination 1: Low-Dimensional, Exact Searches**

- **`vectorIndexType`**: `flat`
- **`datatype`**: `float32`
- **`dimensions`**: ≤ 505
- **`distanceFunction`**: `cosine`

**Real-World Scenario:**

- **Small-Scale Text Classification**: A startup builds a news categorization tool using word embeddings (300 dimensions). Exact cosine similarity searches ensure accurate article tagging without the overhead of approximate methods.

---

#### **Combination 2: High-Dimensional, Performance-Critical Applications**

- **`vectorIndexType`**: `diskANN`
- **`datatype`**: `float32`
- **`dimensions`**: 768 - 1536
- **`distanceFunction`**: `cosine` or `dot product`

**Real-World Scenario:**

- **Real-Time Recommendations**: A streaming service uses user and content embeddings (1024 dimensions) to provide instantaneous movie recommendations. DiskANN accelerates search times, offering a smooth user experience despite the large dataset.

---

#### **Combination 3: Storage-Efficient High-Dimensional Data**

- **`vectorIndexType`**: `quantizedFlat`
- **`datatype`**: `uint8` or `int8`
- **`dimensions`**: 2048
- **`distanceFunction`**: `cosine`

**Real-World Scenario:**

- **Mobile Visual Search**: An app allows users to search for products by uploading photos. High-dimensional image embeddings are quantized to fit the storage constraints of mobile devices, and approximate searches provide quick results.

---

#### **Combination 4: Precision-Critical Scientific Computing**

- **`vectorIndexType`**: `flat`
- **`datatype`**: `float32`
- **`dimensions`**: 4096
- **`distanceFunction`**: `euclidean`

**Real-World Scenario:**

- **Genomic Data Analysis**: Researchers analyze genetic sequences represented as high-dimensional vectors. Precise Euclidean distance calculations are essential for identifying genetic similarities and mutations.

---

#### **Combination 5: Medium-Dimensional Data with Storage Constraints**

- **`vectorIndexType`**: `quantizedFlat`
- **`datatype`**: `uint8`
- **`dimensions`**: 500
- **`distanceFunction`**: `dot product`

**Real-World Scenario:**

- **IoT Sensor Data**: A network of sensors generates medium-dimensional vectors representing environmental data. Quantization reduces storage and transmission costs, and dot product calculations help in identifying patterns and anomalies efficiently.

## Implementation using Financial Datasets

### Setup Containers


1. **`CompanyData`**

    - **Data Types**: `BasicCompanyInfo`, `CompanyOfficer`
    - **Partition Key**: `/Cik`
    - **Id**: `/Cik` to ensure there is only 1 basic information document per company
    - **Vector Paths**: ``
    - **Notes**:
        - **Optimized for Company Queries**: Facilitates queries and reports scoped to specific companies.
        - **Rationale**: Embedding reduces the need for cross-partition queries and improves read performance when retrieving company information along with its officers.


2. **`FinancialFilings`**

    - **Data Types**: `Form10KSection`, `Form13D`
    - **Partition Key**: `/Cik`
    - **Id**: `
    - **Indexing**:
        - **Enable Vector Indexing**: For `Form10KSection` embeddings.
    - **Notes**:
        - **Efficient Semantic Search**: Supports AI-driven searches over financial filings.

3. **`MarketData`**

    - **Data Types**: `DailyMarketData`
    - **Partition Key**: `/Symbol`
    - **Notes**:

        - **High Write Throughput**: Allocate sufficient RU/s to handle frequent updates.
4. **`Holdings`**

    - **Data Types**: `Form13FHolding`
    - **Partition Key**: `/Cusip`
    - **Alternate Partition Key**: `/ManagerName` if queries are more often by manager.
    - **Notes**:

        - **Facilitates Cross-Company Queries**: Efficiently retrieve holdings data for reports.
5. **`NewsArticles`**

    - **Data Types**: `NewsArticle`
    - **Partition Key**: `/PublishDate` (e.g., formatted as `yyyy-MM` for monthly partitions)
    - **Indexing**:

        - **Enable Vector Indexing**: For `ArticleText` embeddings.
    - **Notes**:

        - **Time-Based Partitioning**: Improves performance for time-bound queries.


### Classes
- Every document that will have vector search has only 1 embedding field


In [25]:
#pragma warning disable SKEXP0001,SKEXP0020

/**
* Just for reference, Semantic Kernel has abstracted concepts around vector databases for AI RAG applications.
* This solution could be built using annotations instead and the underlying DB could be changed without changing the application code.
public record 10KDocument(
    [property: VectorStoreRecordKey] string HotelId,
    [property: VectorStoreRecordData] string HotelName,
    [property: VectorStoreRecordData] string Description,
    [property: VectorStoreRecordVector(Dimensions: 4, IndexKind: IndexKind.Hash, DistanceFunction: DistanceFunction.CosineSimilarity), JsonPropertyName("description_embeddings")] ReadOnlyMemory<float>? DescriptionEmbeddings);

*/
[AttributeUsage(AttributeTargets.Class, Inherited = false, AllowMultiple = false)]
public class VectorStoreEntityAttribute: Attribute
{
    public string CollectionName { get; init; } = CollectionName;
}

[AttributeUsage(AttributeTargets.Property, Inherited = false, AllowMultiple = false)]
public sealed class VectorStoreIdAttribute(): Attribute;

[AttributeUsage(AttributeTargets.Property, Inherited = false, AllowMultiple = false)]
public sealed class VectorStorePartitionKeyAttribute(): Attribute;

[AttributeUsage(AttributeTargets.Property, Inherited = false, AllowMultiple = false)]
public sealed class VectorStoreDocumentTypeAttribute(): Attribute;

[AttributeUsage(AttributeTargets.Property, Inherited = false, AllowMultiple = false)]
public sealed class VectorStoreEmbeddingAttribute(): Attribute;

[AttributeUsage(AttributeTargets.Property, Inherited = false, AllowMultiple = false)]
public sealed class VectorStoreEmbeddingDataAttribute(): Attribute;

public abstract record VectorStoreEntity
{
    public string GetContextWindow() => this.GetType().GetProperties(BindingFlags.Public | BindingFlags.Instance)
            .Where(p => p.GetCustomAttribute<VectorStoreEmbeddingAttribute>() != null)
            .Select(p => {
                var value = p.GetValue(this);
                return value switch
                {
                    string stringValue => stringValue,
                    IEnumerable<string> stringList => stringList.Aggregate(new StringBuilder(), (sb, value) => sb.AppendLine(value), sb => sb.ToString()),
                    _ => string.Empty
                };
            })
            .Aggregate(new StringBuilder(), (sb, value) => sb.AppendLine(value), sb => sb.ToString());

    public Task<ReadOnlyMemory<float>> GetEmbedding(ITextEmbeddingGenerationService textEmbeddingService, CancellationToken cancellationToken = default)
    {
        var embeddingString = this.GetContextWindow();
        return textEmbeddingService.GenerateEmbeddingAsync(value: embeddingString, cancellationToken: cancellationToken);
    }

    public int GetTokenCount(Tokenizer s_tokenizer) => s_tokenizer.CountTokens(this.GetContextWindow());

    public PartitionKey GetPartitionKey() => this.GetType().GetProperties(BindingFlags.Public | BindingFlags.Instance)
        .Where(p => p.GetCustomAttribute<VectorStorePartitionKeyAttribute>() != null)
        .Select(p => p.GetValue(this) as string ?? string.Empty)
        .Where(value => !string.IsNullOrWhiteSpace(value))
        .Aggregate(new PartitionKeyBuilder(), (pk, value) => pk.Add(value), pk => pk.Build());  


};

/**
    * Container: Cache
*/
[VectorStoreEntity(CollectionName = "cache")]
public record CacheItem(
    [property: VectorStoreId] string Id, 
    [property: VectorStorePartitionKey] string Type, 
    [property: VectorStoreEmbeddingData] string Prompts,
    string Completion, 
    [property: VectorStoreEmbedding] ReadOnlyMemory<float> Embedding): VectorStoreEntity
{
    public int Ttl => CalculateTtl();
    public int CacheHits { get; set; } = 1; // Start with 1 to avoid immediate eviction
    public DateTime CreatedAt { get; set; } = DateTime.UtcNow;
    public void RegisterHit()
    {
        CacheHits++;
    }

    private int CalculateTtl()
    {
        var elapsedTime = DateTime.UtcNow - CreatedAt;
        var baseTtl = (int)TimeSpan.FromHours(1).TotalSeconds;
        return CacheHits * baseTtl - (int)elapsedTime.TotalSeconds;
    }
}

// Container Properties
public static ContainerProperties getSemanticCacheContainerProperties() {
    var properties = new ContainerProperties(id: "cache", partitionKeyPath: "/type"){
            IndexingPolicy = new (){
                VectorIndexes = new ()
                {
                    new VectorIndexPath(){
                        Path = "/embedding",
                        Type = VectorIndexType.DiskANN
                    }
                },
            },
            VectorEmbeddingPolicy = new VectorEmbeddingPolicy(new (){
                new Embedding(){
                    Path = "/embedding",
                    Dimensions = 1536,
                    DataType = VectorDataType.Float32,
                    DistanceFunction = Microsoft.Azure.Cosmos.DistanceFunction.Cosine
                }
            }),
            DefaultTimeToLive = (int)TimeSpan.FromDays(1).TotalSeconds
    };
    properties.IndexingPolicy.IncludedPaths.Add(new () { Path = "/*" });

    return properties;
} 

/**
    * Container: ChatThreads
*/

// Using Semantic Kernel's ChatHistory class as a building block for ChatThreads.
[VectorStoreEntity(CollectionName = "chat")]
public record ChatThreadMessage(
    [property: VectorStoreId] string Id, 
    [property: VectorStorePartitionKey] string UserId, 
    [property: VectorStorePartitionKey] string ThreadId, 
    ChatMessageContent MessageContent, 
    int Tokens = 0)
{
    public string Type { get; set; } = "ChatThreadMessage";
    public bool Deleted { get; set; } = false;

    public DateTime CreatedAt { get; set; } = DateTime.UtcNow;
    public DateTime LastUpdatedAt { get; set; } = DateTime.UtcNow;

    [VectorStoreEmbeddingData,JsonIgnore]
    public string ContextWindow => MessageContent.Content;
    public static implicit operator ChatMessageContent(ChatThreadMessage v) => v.MessageContent;
};

[VectorStoreEntity(CollectionName = "chat")]
public record ChatThread(
    [property: VectorStorePartitionKey] string UserId, 
    [property: VectorStorePartitionKey] string ThreadId, 
    string DisplayName = "New Chat", 
    int? Tokens = 0)
{
    public string Type { get; set; } = "ChatThread";
    
    [VectorStoreId]
    public string Id { get; set; } = ThreadId;

    public bool Deleted { get; set; } = false;
    public DateTime CreatedAt { get; set; } = DateTime.UtcNow;
    public DateTime LastUpdatedAt { get; set; } = DateTime.UtcNow;

    [JsonIgnore]
    public IEnumerable<ChatThreadMessage> Messages { get; set; } = new List<ChatThreadMessage>();

    [VectorStoreEmbeddingData,JsonIgnore]
    public string ContextWindow => Messages.Select(m => m.ContextWindow).Aggregate(new StringBuilder(), (sb, value) => sb.AppendLine(value), sb => sb.ToString());

    public IEnumerable<string> GetMessageContentsForRole(AuthorRole role) => Messages.Where(x => x.MessageContent.Role == role).Select(x => x.MessageContent.Content);
    public IEnumerable<string> GetAllMessageContents() => Messages.Select(x => x.MessageContent.Content);
    public static implicit operator ChatHistory(ChatThread v) => new ChatHistory(messages: v.Messages.Select(m => (ChatMessageContent)m));
};


// Container Properties
public static ContainerProperties getChatThreadContainerProperties() {
    var properties = new ContainerProperties(id: "chat", partitionKeyPaths: new Collection<string>(){ "/userId", "/threadId" });
    properties.IndexingPolicy.ExcludedPaths.Add(new () { Path = "/*" });

    foreach (var path in new string[] { "/userId/?", "/threadId/?"})
    {
        properties.IndexingPolicy.IncludedPaths.Add(new () { Path = path });
    }

    return properties;
}


/**
    * Container: CompanyInfo
*/

public record CompanyOfficer(
    string FirstName,
    string LastName,
    int? Age,
    string Title,
    int? YearBorn,
    long TotalPay
);

public record SecurityListing(
    string Cusip, 
    string Name,  
    string Exchange, 
    string Symbol, 
    string IsinNumber);

[VectorStoreEntity(CollectionName = "companyInfo")]
public record CompanyInfo(
    int Cik,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string Sector,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string Industry,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string SubIndustry,
    [property: VectorStoreEmbeddingData] string Cusip6,
    string Lei,
    string CompanyName,
    string Address1,
    string City,
    string State,
    string Zip,
    string Country,
    string Phone,
    string Website,
    [property: VectorStoreEmbeddingData] string LongBusinessSummary,
    [property: VectorStoreEmbeddingData] List<string> ReferenceNames,
    [property: VectorStoreEmbeddingData] List<string> Tickers,
    List<CompanyOfficer> CompanyOfficers,
    List<SecurityListing> SecurityListings,
    string WebsiteUrl,
    [property: VectorStoreEmbeddingField] ReadOnlyMemory<float>? Embedding = null
){
    public string Type { get; set; } = "CompanyInfo";

    [VectorStoreId]
    public string Id { get; set; } = Cik.ToString();

    [VectorStorePartitionKey]
    public string PartitionKey { get; set; } = Cik.ToString();
};

[VectorStoreEntity(CollectionName = "companyInfo")]
public record Form10KSection(
    int Cik,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string Sector,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string Industry,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string SubIndustry,
    string SequenceId, 
    DateTime FilingDate, 
    string SectionName, 
    string SectionShortName, 
    [property: VectorStoreEmbeddingData] string SectionText, 
    [property: VectorStoreEmbeddingField] ReadOnlyMemory<float>? Embedding = null, 
    string Type = "10-K-section"){

    [VectorStoreId]
    public string Id { get; set; } = $"{Cik}_{FilingDate:yyyy-MM-dd}_{Type}_{SectionName}";

    [VectorStorePartitionKey]
    public string PartitionKey { get; set; } = Cik.ToString();

    [Description("The CIK of the manager who filed the SEC document.")]
    public string FilerCik { get; set; } = Cik.ToString();

    [JsonIgnore, Description("The Accession Number is a unique identifier assigned by the SEC to each filing.")]
    public string AccessionNumber => $"{FilerCik.ToString().PadLeft(10, '0')}-{FilingDate:yy}-{SequenceId.PadLeft(4, '0')}";

    [JsonIgnore]
    public Uri SourceUri => new Uri($"https://www.sec.gov/Archives/edgar/data/{Cik}/{AccessionNumber}.txt");
}

[VectorStoreEntity(CollectionName = "companyInfo")]
public record Form13D(
    int Cik, 
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string Sector,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string Industry,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string SubIndustry,
    string SequenceId, 
    string ReportingPerson, 
    DateTime FilingDate, 
    string Description, 
    ReadOnlyMemory<float>? Embedding = null, 
    string Type = "13D"){

    [VectorStoreId]
    public string Id { get; set; } = $"{Cik}_{FilingDate:yyyy-MM-dd}_{Type}";

    [VectorStorePartitionKey]
    public string PartitionKey { get; set; } = Cik.ToString();

    [Description("The CIK of the manager who filed the SEC document.")]
    public string FilerCik { get; set; } = Cik.ToString();

    [JsonIgnore, Description("The Accession Number is a unique identifier assigned by the SEC to each filing.")]
    public string AccessionNumber => $"{FilerCik.ToString().PadLeft(10, '0')}-{FilingDate:yy}-{SequenceId.PadLeft(4, '0')}";

    [JsonIgnore]
    public Uri SourceUri => new Uri($"https://www.sec.gov/Archives/edgar/data/{Cik}/{AccessionNumber}.txt");
}

[VectorStoreEntity(CollectionName = "companyInfo")]
public record Form13FHR(
    int Cik, 
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string Sector,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string Industry,
    [property: VectorStoreEmbeddingData, VectorStorePartitionKey] string SubIndustry,
    string SequenceId, 
    string Cusip,  
    DateTime FilingDate, 
    int ManagerCik, 
    string ManagerName, 
    string SecurityName, 
    int Shares, 
    int Value, 
    string SecurityType, 
    ReadOnlyMemory<float>? Embedding = null, 
    string Type = "13F-HR"){
    [VectorStoreId]
    public string Id { get; set; } = $"{Cik}_{FilingDate:yyyy-MM-dd}_{Type}";

    [VectorStorePartitionKey]
    public string PartitionKey { get; set; } = Cik.ToString();

    [Description("The CIK of the manager who filed the SEC document.")]
    public string FilerCik { get; set; } = Cik.ToString();

    [JsonIgnore, Description("The Accession Number is a unique identifier assigned by the SEC to each filing.")]
    public string AccessionNumber => $"{FilerCik.ToString().PadLeft(10, '0')}-{FilingDate:yy}-{SequenceId.PadLeft(4, '0')}";

    [JsonIgnore]
    public Uri SourceUri => new Uri($"https://www.sec.gov/Archives/edgar/data/{Cik}/{AccessionNumber}.txt");
}

// Container Properties
public static ContainerProperties getCompanyInfoContainerProperties() {
    var properties = new ContainerProperties(id: "companyInfo", partitionKeyPath: "/partitionKey"){
            IndexingPolicy = new (){
                VectorIndexes = new ()
                {
                    new VectorIndexPath(){
                        Path = "/embedding",
                        Type = VectorIndexType.DiskANN
                    }
                },
            },
            VectorEmbeddingPolicy = new VectorEmbeddingPolicy(new (){
                new Embedding(){
                    Path = "/embedding",
                    Dimensions = 1536,
                    DataType = VectorDataType.Float32,
                    DistanceFunction = Microsoft.Azure.Cosmos.DistanceFunction.Cosine
                }
            })
    };
    properties.IndexingPolicy.ExcludedPaths.Add(new () { Path = "/embedding/*" });
    properties.IndexingPolicy.ExcludedPaths.Add(new () { Path = "/sectionText/*" });
    properties.IndexingPolicy.ExcludedPaths.Add(new () { Path = "/longBusinessSummary/*" });

    return properties;
} 

/**
    * Container: MarketData
*/
public record DailyStockMarketReport(int Cik, string Symbol, DateTime Date, float Open, float High, float Low, float Close, long Volume) {
    public string Type => "DailyMarketData";
    public string Id { get; set; } = $"{Symbol}_{Date:yyyy-MM-dd}";
    public string PartitionKey { get; set; } = Symbol;
}

public record NewsArticle(string Headline, string ArticleText, string SourceName, string Uri, DateTime PublishDate){
    public string Type => "NewsArticle";
    public string Id => $"{SourceName}_{Uri}";
    public string PartitionKey { get; set; } = SourceName;
}

public static ContainerProperties getMarketDataContainerProperties() {
    var properties = new ContainerProperties(id: "marketData", partitionKeyPath: "/partitionKey"){
            IndexingPolicy = new (){
                VectorIndexes = new ()
                {
                    new VectorIndexPath(){
                        Path = "/embedding",
                        Type = VectorIndexType.DiskANN
                    }
                },
            },
            VectorEmbeddingPolicy = new VectorEmbeddingPolicy(new (){
                new Embedding(){
                    Path = "/embedding",
                    Dimensions = 1536,
                    DataType = VectorDataType.Float32,
                    DistanceFunction = Microsoft.Azure.Cosmos.DistanceFunction.Cosine
                }
            })
    };
    properties.IndexingPolicy.ExcludedPaths.Add(new () { Path = "/embedding/*" });
    properties.IndexingPolicy.ExcludedPaths.Add(new () { Path = "/sectionText/*" });
    properties.IndexingPolicy.ExcludedPaths.Add(new () { Path = "/longBusinessSummary/*" });

    return properties;
} 

var Form10KSectionDescriptions = new Dictionary<string, string>
{
    { "item1", "Business: requires a description of the company’s business, including its main products and services, what subsidiaries it owns, and what markets it operates in" },
    { "item1a", "Risk Factors: includes information about the most significant risks that apply to the company or to its securities" },
    { "item1b", "Unresolved Staff Comments: requires the company to explain certain comments it has received from the SEC staff on previously filed reports that have not been resolved after an extended period of time" },
    { "item2", "Properties: includes information about the company’s significant properties, such as principal plants, mines and other materially important physical properties" },
    { "item3", "Legal Proceedings: requires the company to include information about significant pending lawsuits or other legal proceedings, other than ordinary litigation" },
    { "item7", "Management’s Discussion and Analysis of Financial Condition and Results of Operations (MD&A): gives the company’s perspective on the business results of the past financial year. This section, known as the MD&A for short, allows company management to tell its story in its own words" },
    { "item7a", "Quantitative and Qualitative Disclosures About Market Risk: requires information about the company’s exposure to market risk, such as interest rate risk, foreign currency exchange risk, commodity price risk or equity price risk" },
    { "item8", "Financial Statements and Supplementary Data: requires the company’s audited financial statements" },
    { "item10", "Directors, Executive Officers and Corporate Governance: requires information about the background and experience of the company’s directors and executive officers, the company’s code of ethics, and certain qualifications for directors and committees of the board of directors" },
    { "item11", "Executive Compensation: includes detailed disclosure about the company’s compensation policies and programs and how much compensation was paid to the top executive officers of the company in the past year" },
    { "item15", "Exhibits, Financial Statement Schedules: Many exhibits are required, including documents such as the company’s bylaws, copies of its material contracts, and a list of the company’s subsidiaries" }
};

// public class SemanticTextMemory(AzureCosmosDBNoSQLVectorStoreRecordCollection<CacheItem> vectorStore): IMemoryStore
// {
//     public Task CreateCollectionAsync(string collectionName, CancellationToken cancellationToken = default)
//     {
//         return vectorStore.CreateCollectionAsync(cancellationToken);
//     }

//     public IAsyncEnumerable<string> GetCollectionsAsync(CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }

//     public Task<bool> DoesCollectionExistAsync(string collectionName, CancellationToken cancellationToken = default)
//     {
//         return vectorStore.CollectionExistsAsync(cancellationToken);
//     }

//     public Task DeleteCollectionAsync(string collectionName, CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }


//     public Task<string> UpsertAsync(string collectionName, MemoryRecord record, CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }

//     public IAsyncEnumerable<string> UpsertBatchAsync(string collectionName, IEnumerable<MemoryRecord> records, CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }

//     public Task<MemoryRecord?> GetAsync(string collectionName, string key, bool withEmbedding = false, CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }

//     public IAsyncEnumerable<MemoryRecord> GetBatchAsync(string collectionName, IEnumerable<string> keys, bool withEmbeddings = false, CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }

//     public Task RemoveAsync(string collectionName, string key, CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }

//     public Task RemoveBatchAsync(string collectionName, IEnumerable<string> keys, CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }

//     public IAsyncEnumerable<(MemoryRecord, double)> GetNearestMatchesAsync(
//         string collectionName,
//         ReadOnlyMemory<float> embedding,
//         int limit,
//         double minRelevanceScore = 0.0,
//         bool withEmbeddings = false,
//         CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }

//     public Task<(MemoryRecord, double)?> GetNearestMatchAsync(
//         string collectionName,
//         ReadOnlyMemory<float> embedding,
//         double minRelevanceScore = 0.0,
//         bool withEmbedding = false,
//         CancellationToken cancellationToken = default)
//     {
//         throw new NotImplementedException();
//     }
// }
public class VectorStore
{
    private readonly Database _database;
    private readonly string[] _containers;
    public VectorStore(Database database, string[] containers)
    {
        _database = database;
        _containers = containers;
    }

    public VectorCollection<TRecord> GetContainer<TRecord>(string containerName)
    {
        var container = _database.GetContainer(containerName);
        return new VectorCollection<TRecord>(container);
    }

    public async IAsyncEnumerable<string> ListCollectionNamesAsync([EnumeratorCancellation] CancellationToken cancellationToken = default)
    {
        const string Query = "SELECT VALUE(c.id) FROM c";

        using var feedIterator = this._database.GetContainerQueryIterator<string>(Query);

        while (feedIterator.HasMoreResults)
        {
            var next = await feedIterator.ReadNextAsync(cancellationToken).ConfigureAwait(false);

            foreach (var containerName in next.Resource)
            {
                yield return containerName;
            }
        }
    }
}

public class VectorCollection<TRecord>
{
    private readonly Container _container;

    public VectorCollection(Container container)
    {
        _container = container;
    }

    public async Task UpsertAsync(TRecord item)
    {
        await _container.UpsertItemAsync(item);
    }

    public async Task<TRecord> GetAsync(string id, string partitionKey)
    {
        return await _container.ReadItemAsync<TRecord>(id, new PartitionKey(partitionKey));
    }

    // public async Task RemoveAsync(string id)
    // {
    //     await _container.DeleteItemAsync<TRecord>(id);
    // }
    
    public IAsyncEnumerable<(TRecord, double)> GetNearestMatchesAsync(
        ReadOnlyMemory<float> embedding,
        int limit,
        double minRelevanceScore = 0.0,
        bool withEmbeddings = false,
        CancellationToken cancellationToken = default)
    {
        throw new NotImplementedException();
    }

    public Task<(TRecord, double)?> GetNearestMatchAsync(
        ReadOnlyMemory<float> embedding,
        double minRelevanceScore = 0.0,
        bool withEmbedding = false,
        CancellationToken cancellationToken = default)
    {
        throw new NotImplementedException();
    }
}
public class CosmosCache
{
    private readonly Container _container;
    private readonly Database _database;
    private List<string> _fields = new List<string> { "id", "type", "prompts", "completion", "embedding" };
    private List<string> _collections = new List<string> { "cache" };

    public CosmosCache(Database database, Container container)
    {
        _database = database;
        _container = container;
    }

    public async Task UpsertAsync(CacheItem item)
    {
        await _container.UpsertItemAsync(item, new PartitionKey(item.Type));
    }

    public async Task<CacheItem> GetAsync(string id)
    {
        return await _container.ReadItemAsync<CacheItem>(id, new PartitionKey(id));
    }

    public async Task RemoveAsync(string id)
    {
        await _container.DeleteItemAsync<CacheItem>(id, new PartitionKey(id));
    }
    
    public IAsyncEnumerable<(CacheItem, double)> GetNearestMatchesAsync(
        ReadOnlyMemory<float> embedding,
        int limit,
        double minRelevanceScore = 0.0,
        bool withEmbeddings = false,
        CancellationToken cancellationToken = default)
    {
        throw new NotImplementedException();
    }

    public Task<(CacheItem, double)?> GetNearestMatchAsync(
        ReadOnlyMemory<float> embedding,
        double minRelevanceScore = 0.0,
        bool withEmbedding = false,
        CancellationToken cancellationToken = default)
    {
        throw new NotImplementedException();
    }
    // private QueryDefinition GetSelectQuery(List<AzureCosmosDBNoSQLCompositeKey> keys, List<string> fields)
    // {
    //     // Verify.True(keys.Count > 0, "At least one key should be provided.", nameof(keys));

    //     const string WhereClauseDelimiter = " OR ";
    //     const string SelectClauseDelimiter = ",";

    //     const string RecordKeyVariableName = "rk";
    //     const string PartitionKeyVariableName = "pk";

    //     const string TableVariableName = "x";

    //     var selectClauseArguments = string.Join(SelectClauseDelimiter,
    //         fields.Select(field => $"{TableVariableName}.{field}"));

    //     var whereClauseArguments = string.Join(WhereClauseDelimiter,
    //         keys.Select((key, index) =>
    //             $"({TableVariableName}.{this._keyStoragePropertyName} = @{RecordKeyVariableName}{index} AND " +
    //             $"{TableVariableName}.{this._partitionKeyStoragePropertyName} = @{PartitionKeyVariableName}{index})"));

    //     var query = $"SELECT {selectClauseArguments} FROM {TableVariableName} WHERE {whereClauseArguments}";

    //     var queryDefinition = new QueryDefinition(query);

    //     for (var i = 0; i < keys.Count; i++)
    //     {
    //         var recordKey = keys[i].RecordKey;
    //         var partitionKey = keys[i].PartitionKey;

    //         // Verify.NotNullOrWhiteSpace(recordKey);
    //         // Verify.NotNullOrWhiteSpace(partitionKey);

    //         queryDefinition.WithParameter($"@{RecordKeyVariableName}{i}", recordKey);
    //         queryDefinition.WithParameter($"@{PartitionKeyVariableName}{i}", partitionKey);
    //     }

    //     return queryDefinition;
    // }
}
public class RagContext
{
    // https://github.com/microsoft/semantic-kernel/discussions/5815
    // https://github.com/microsoft/semantic-kernel/blob/382972b666fb1d1b843b1a5b262f8a960dabb9a4/dotnet/src/SemanticKernel.UnitTests/AI/ChatCompletion/ChatHistoryTests.cs#L17-L45
    public ChatHistory ChatHistory { get; set; } = new ChatHistory();

    private Container _chatThreadContainer;
    private Container _cacheContainer;
    private Container _companyInfoContainer;
    private Container _marketDataContainer;
    
    private readonly CosmosClient _cosmosClient;
    private readonly Database _database;

    private string _contextId {get; set;}

    public RagContext(Database database, CosmosClient cosmosClient)
    {
        _cosmosClient = cosmosClient;
        _database = database;
        _chatThreadContainer = _database.GetContainer("chatThread");
        _cacheContainer = _database.GetContainer("cache");
        _companyInfoContainer = _database.GetContainer("companyInfo");
        _marketDataContainer = _database.GetContainer("marketData");
    }

    public async Task UpsertCacheAsync(CacheItem item)
    {
        await _cacheContainer.UpsertItemAsync(item, new PartitionKey(item.Type));
    }

    public async Task<CacheItem> GetCacheAsync(string id)
    {
        return await _cacheContainer.ReadItemAsync<CacheItem>(id, new PartitionKey(id));
    }

    public async Task RemoveCacheAsync(string id)
    {
        await _cacheContainer.DeleteItemAsync<CacheItem>(id, new PartitionKey(id));
    }
}


In [23]:
var t = new CacheItem("test", "testtype", "test", "test", new ReadOnlyMemory<float>(new float[]{1,2,3,4}));
Console.WriteLine(t.Ttl);
t.RegisterHit();
Console.WriteLine(t.Ttl);
var t2 = new List<string> { "id", "type", "prompts", "completion", "embedding" };
Console.WriteLine(t2.ToString());
// public string data = """
// {
//   "Cik": 320193,
//   "FilingDate": "2021-09-30T00:00:00",
//   "SectionName": "item1",
//   "SectionShortName": "Business",
//   "SequenceId": "1",
//   "SectionText": "asdljkdjadsk",
//   "Id": "test",
//   "Type": "testtype"
// }
// """;
// var test = new Form10KSection(320193, "1", DateTime.Parse("2021-09-30"), "item1", "Business", "asdljkdjadsk");

// var deserialized = JsonSerializer.Deserialize<Form10KSection>(data);
// Console.WriteLine(deserialized);
// var serialized = JsonSerializer.Serialize(test);

// Console.WriteLine(serialized);

3600
7200
System.Collections.Generic.List`1[System.String]


In [4]:
using System;

/**
https://github.com/microsoft/semantic-kernel/blob/382972b666fb1d1b843b1a5b262f8a960dabb9a4/dotnet/src/SemanticKernel.UnitTests/AI/ChatCompletion/ChatHistoryTests.cs#L17-L45
*/
public class CosmosWrapper
{
    private CosmosNoSqlService CosmosService;
    public Database DatabaseClient;
    public const string DEFAULT_PARTITION_KEY_PATH = "/PartitionKey";

    public CosmosWrapper()
    {
        this.CosmosService = new CosmosNoSqlService();
        this.DatabaseClient = this.CosmosService.databaseClient;
    }   

    public async Task<Container> GetOrCreateContainer(string containerName)
    {   var containerProperties = new ContainerProperties(id: containerName, partitionKeyPath: DEFAULT_PARTITION_KEY_PATH){
            DefaultTimeToLive = (int)TimeSpan.FromDays(1).TotalSeconds, 
            VectorEmbeddingPolicy = new VectorEmbeddingPolicy(new Collection<Embedding>(){
                    new Embedding(){
                        Path = "ContentEmbedding",
                        Dimensions = 1536,
                        DataType = VectorDataType.Float32,
                        DistanceFunction = Microsoft.Azure.Cosmos.DistanceFunction.Cosine
                    }
                }),
                IndexingPolicy = new IndexingPolicy(){
                    VectorIndexes = new ()
                    {
                        new VectorIndexPath(){
                            Path = "/ContentEmbedding",
                            Type = VectorIndexType.DiskANN,
                        }
                    }
                }
        };
        containerProperties.IndexingPolicy.IncludedPaths.Add(new IncludedPath { Path = "/*" });
        containerProperties.IndexingPolicy.ExcludedPaths.Add(new ExcludedPath { Path = "/\"_etag\"/?" });
        var c = await this.DatabaseClient.CreateContainerIfNotExistsAsync(containerProperties);
        return c.Container;
    }
}

Error: (12,17): error CS0103: The name 'Duration' does not exist in the current context

# Semantic Search on Cosmos DB

# Structured Database Copilot
NL2SQL - Database query generation

### Considerations
- Usually good at building most of the database query, however it needs prompt tuning or native functions to improve the where clause.

**Example User Stories**
- I have application monitoring or metric data that I want to derive insights from. 
- I want to chat over the entire corpus of Service Now or other ICM support ticket information